In [ ]:
!pip install gcn-kafka

In [1]:
from gcn_kafka import Consumer

# For Testing purposes

I will only fetch messages in a single day for testing the script

How can I search for messages occurring within a given date range?

To search for messages in a given date range, you can use the offsets_for_times() function from the Consumer class to get the message offsets for the desired date range. You can then assign the starting offset to the Consumer and read the desired number of messages. When doing so, keep in mind that the stream buffers are finite in size. It is not possible to recover messages prior to the start of the stream buffer. The GCN stream buffers are currently set to hold messages from the past few days.

In [2]:
import datetime
from gcn_kafka import Consumer
from confluent_kafka import TopicPartition

In [4]:
# Connect as a consumer (client "MMA-app-Diaspora")
# Warning: don't share the client secret with others.
consumer = Consumer(client_id='3f4l3qp0r2no2o4vnik7eg7ut0',
                    client_secret='136doan9ouhbo0sl30r9h6s0d56ucrik58lcv9egl3c9e0i2s1o')

%3|1730936115.193|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://kafka.gcn.nasa.gov:9092/bootstrap]: sasl_ssl://kafka.gcn.nasa.gov:9092/bootstrap: Failed to connect to broker at [2600:1f10:4f61:4a00:9f10:6cf4:5ba3:5812]:9092: Network is unreachable (after 3ms in state CONNECT)
%3|1730936117.191|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://kafka.gcn.nasa.gov:9092/bootstrap]: sasl_ssl://kafka.gcn.nasa.gov:9092/bootstrap: Failed to connect to broker at [2600:1f10:4f61:4a00:9f10:6cf4:5ba3:5812]:9092: Network is unreachable (after 1ms in state CONNECT, 1 identical error(s) suppressed)
%3|1730936117.587|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: kafka1.gcn.nasa.gov:9092: Failed to connect to broker at [2600:1f10:4f61:4a00:79d8:1a49:534a:ee9]:9092: Network is unreachable (after 1ms in state CONNECT)


In [ ]:
# get messages occurring 1 days ago
timestamp1 = int((datetime.datetime.now() - datetime.timedelta(days=3)).timestamp() * 1000)
timestamp2 = timestamp1 + 86400000 # +1 day

# I selected following topic for testing purpose
# Subscribe to topics and receive alerts
# consumer.subscribe(['gcn.classic.text.FERMI_GBM_ALERT'])
# topic = 'gcn.classic.voevent.INTEGRAL_SPIACS'
# topic = 'gcn.classic.text.FERMI_GBM_ALERT'

topic = 'gcn.classic.text.GRB_FINAL'

start = consumer.offsets_for_times(
    [TopicPartition(topic, 0, timestamp1)])
end = consumer.offsets_for_times(
    [TopicPartition(topic, 0, timestamp2)])

### Ran into following error in both cells. When I run again, the error goes away

%3|1729038170.336|FAIL|rdkafka#consumer-2| [thrd:sasl_ssl://kafka1.gcn.nasa.gov:9092/1]: sasl_ssl://kafka1.gcn.nasa.gov:9092/1: Failed to connect to broker at [2600:1f10:4f61:4a00:79d8:1a49:534a:ee9]:9092: Network is unreachable (after 70ms in state CONNECT)
%3|1729038171.337|FAIL|rdkafka#consumer-2| [thrd:sasl_ssl://kafka1.gcn.nasa.gov:9092/1]: sasl_ssl://kafka1.gcn.nasa.gov:9092/1: Failed to connect to broker at [2600:1f10:4f61:4a00:79d8:1a49:534a:ee9]:9092: Network is unreachable (after 1ms in state CONNECT, 1 identical error(s) suppressed)

SOlution:
'retries': 5,  # Set retries in case of connection issues
Connection Retry Logic: It’s good practice to include retry logic in your connection code to handle such intermittent issues. For example, you can include retries with exponential backoff so that your application tries to reconnect a few times before failing.

In [ ]:
import pprint

In [ ]:
consumer.assign(start)
for message in consumer.consume(end[0].offset - start[0].offset, timeout=1):
    print("\n message1: \n")
    # print(message.value())
    print(message)

## Messaage object has following fields

https://docs.confluent.io/platform/current/clients/confluent-kafka-python/html/index.html#confluent_kafka.Message

In [ ]:
consumer.assign(start)
for message in consumer.consume(end[0].offset - start[0].offset, timeout=1):
    print("\n message1: \n")
    # print(message.value())
    print(message.offset())
    print(message.topic())
    print(message.value()) 

## message.value()

chatgpt

To print the output more cleanly with newlines, you can decode the byte string to a regular string and replace the \n characters with actual newlines. Here's how you can modify your code:

python
Copy code
<code>
consumer.assign(start)
for message in consumer.consume(end[0].offset - start[0].offset, timeout=1):
    print("\nMessage:\n")
    # Decode the byte message to a string
    decoded_message = message.value().decode('utf-8')  # Decodes bytes to string
    # Print the message with actual newlines
    print(decoded_message)
</code>
Explanation:
message.value() gives a byte string (as indicated by the b'').
.decode('utf-8') converts it into a human-readable string.
Since the string already contains \n, it will automatically be printed as newlines in the output.
This should print the output cleanly, without \n literals.

In [ ]:
consumer.assign(start)
for message in consumer.consume(end[0].offset - start[0].offset, timeout=1):
    print("\nMessage:\n")
    
    # Decode the byte message to a string
    decoded_message = message.value().decode('utf-8')  # Decodes bytes to string
    
    # Print the message with actual newlines
    print(decoded_message)

In [3]:
# Connect as a consumer (client "MMA-app-Diaspora")
# Warning: don't share the client secret with others.
config = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config=config,
                    client_id='3f4l3qp0r2no2o4vnik7eg7ut0',
                    client_secret='136doan9ouhbo0sl30r9h6s0d56ucrik58lcv9egl3c9e0i2s1o',
                    domain='gcn.nasa.gov')

In [12]:
# Subscribe to topics and receive alerts
consumer.subscribe(['gcn.classic.text.LVC_COUNTERPART',
                    'gcn.classic.text.LVC_EARLY_WARNING',
                    'gcn.classic.text.LVC_INITIAL',
                    'gcn.classic.text.LVC_PRELIMINARY',
                    'gcn.classic.text.LVC_RETRACTION',
                    'gcn.classic.text.LVC_TEST',
                    'gcn.classic.text.LVC_UPDATE'])


In [4]:
# consumer.subscribe(['gcn.classic.voevent.LVC_COUNTERPART',
#                     'gcn.classic.voevent.LVC_EARLY_WARNING',
#                     'gcn.classic.voevent.LVC_INITIAL',
#                     'gcn.classic.voevent.LVC_PRELIMINARY',
#                     'gcn.classic.voevent.LVC_RETRACTION',
#                     'gcn.classic.voevent.LVC_UPDATE'])

consumer.subscribe(['igwn.gwalert'])


In [ ]:
for message in consumer.consume(timeout=1):
    if message.error():
        print(message.error())
        continue
    # Print the topic and message ID
    print(f'topic={message.topic()}, offset={message.offset()}')
    value = message.value().decode('utf-8')
    print(value)

In [19]:
# List of topics to subscribe to
# topics = [
#     'gcn.classic.text.LVC_COUNTERPART',
#     'gcn.classic.text.LVC_EARLY_WARNING',
#     'gcn.classic.text.LVC_INITIAL',
#     'gcn.classic.text.LVC_PRELIMINARY',
#     'gcn.classic.text.LVC_RETRACTION',
#     'gcn.classic.text.LVC_UPDATE'
# ]

# topics = [
#     'gcn.classic.text.LVC_PRELIMINARY',
# ]
# topics = [
#     'igwn.gwalert'
# ]

topics = [
    'gcn.classic.voevent.LVC_PRELIMINARY',
]

# topics =[
#     'gcn.circulars'
# ]
# Define date range (example: last 3 days)
timestamp_start = int((datetime.datetime.now() - datetime.timedelta(days=30)).timestamp() * 1000)
timestamp_end = int((datetime.datetime.now() - datetime.timedelta(days=1)).timestamp() * 1000)  # +1 day

# Loop through each topic
with open("gcn.classic.voevent.LVC_PRELIMINARY_Nov6_past30days.txt", "w") as file:
    for topic in topics:
        print(f"Fetching messages for topic: {topic} within date range.")

        # Get the starting and ending offsets for the desired date range
        start_offsets = consumer.offsets_for_times([TopicPartition(topic, 0, timestamp_start)])
        end_offsets = consumer.offsets_for_times([TopicPartition(topic, 0, timestamp_end)])

        # Assign the starting offset to the consumer for the current topic
        consumer.assign(start_offsets)

        # Calculate the number of messages to read based on the offsets
        num_messages = end_offsets[0].offset - start_offsets[0].offset
        if num_messages <= 0:
            print(f"No messages found in the date range for topic: {topic}")
            continue

        # Consume and print the messages for the topic within the date range
        for message in consumer.consume(num_messages, timeout=1):
            file.write(f"Topic: {topic}, Message: {message.value().decode('utf-8')}")
            # print(f"Topic: {topic}, Message: {message.value().decode('utf-8')}")
            
            
             # Decode the byte message to a string
    #         decoded_message = message.value().decode('utf-8')  # Decodes bytes to string

    #         # Print the message with actual newlines
    #         print(decoded_message)

# # Close the consumer after processing
# consumer.close()

Fetching messages for topic: gcn.classic.voevent.LVC_PRELIMINARY within date range.


%3|1730939388.299|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: kafka1.gcn.nasa.gov:9092: Failed to connect to broker at [2600:1f10:4f61:4a00:79d8:1a49:534a:ee9]:9092: Network is unreachable (after 1ms in state CONNECT)
